<a href="https://colab.research.google.com/github/jm19465/CE888/blob/master/Lab_5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
np.set_printoptions(precision = 3)

df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", delimiter = ",")
df.drop(df.columns[0], axis = 1, inplace = True)

In [0]:
def replace(orig, percentage=0.1):
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
def predict_rating(user_id, item_id):
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating, alpha=0.0001):
    prediction_rating = predict_rating(user_id, item_id)
    error =  prediction_rating - rating
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * error * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * error * user_pref_values
    return error

def sgd(iterations=10):
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not rating == 99.0:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if (iteration % 100) == 0:
        print(mse)

In [73]:
print(df)

       -7.82   8.79  -9.66  -8.16  -7.52  ...  99.22  -5.63  99.23  99.24  99.25
0       4.08  -0.29   6.36   4.37  -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
1      99.00  99.00  99.00  99.00   9.03  ...  99.00  99.00  99.00  99.00  99.00
2      99.00   8.35  99.00  99.00   1.80  ...  99.00  99.00  99.00  99.00  99.00
3       8.50   4.61  -4.17  -5.39   1.36  ...   1.55   3.11   6.55   1.80   1.60
4      -6.17  -3.54   0.44  -8.50  -7.09  ...  -3.35   0.05  -9.08  -5.05  -3.45
...      ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24977   0.44   7.43   9.08   2.33   3.20  ...   9.03   6.55   8.69   8.79   7.43
24978   9.13  -8.16   8.59   9.08   0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24979  99.00  99.00  99.00  99.00  -7.77  ...  99.00  99.00  99.00  99.00  99.00
24980  99.00  99.00  99.00  99.00  -9.71  ...  99.00  99.00  99.00  99.00  99.00
24981   2.43   2.67  -3.98   4.27  -2.28  ...  99.00  99.00  99.00  99.00  99.00

[24982 rows x 100 columns]


In [52]:
arr = df.values
print(arr)

[[ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.    8.35 99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 -3.98 ... 99.   99.   99.  ]]


In [53]:
new_arr, idx = replace(arr, 0.1)
print(new_arr)

[[ 4.08 -0.29  6.36 ... 99.   -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.    8.35 99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.    2.67 -3.98 ... 99.   99.   99.  ]]


In [54]:
arr[idx[0][0], idx[1][0]] # ORIGINAL SET

7.38

In [55]:
new_arr[idx[0][0], idx[1][0]] # VALIDATION SET

99.0

In [64]:
n_latent_factors = 3
user_ratings = new_arr[:,:]


latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
user_ratings


array([[ 4.08, -0.29,  6.36, ..., 99.  , -4.32,  1.07],
       [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
       [99.  ,  8.35, 99.  , ..., 99.  , 99.  , 99.  ],
       ...,
       [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
       [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
       [99.  ,  2.67, -3.98, ..., 99.  , 99.  , 99.  ]])

In [74]:
sgd()

25.85637108331107
25.08000829466745
24.898612200930803
24.74960842647684
24.601788577323166
24.45190641790147
24.299501197663385
24.144524353837618
23.986929294571727
23.82652987486962


In [75]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[0.801, 0.433, 0.392, ..., 0.878, 0.291, 1.233],
       [1.605, 0.862, 0.869, ..., 1.927, 0.597, 2.625],
       [0.813, 0.505, 0.729, ..., 1.099, 0.403, 1.394],
       ...,
       [1.443, 0.839, 0.942, ..., 1.678, 0.607, 2.269],
       [1.104, 0.671, 0.837, ..., 1.323, 0.506, 1.752],
       [1.019, 0.562, 0.521, ..., 1.103, 0.381, 1.549]])

In [0]:
values = [zip(new_arr[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [77]:
comparison_data

,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,-4.76,-8.5.1,-6.75,-7.18,8.45,-7.18.1,-7.52.1,-7.43,-9.81,-9.85.1,-9.85.2,-9.37,1.5,-4.37,-9.81.1,-8.5.2,1.12,7.82,2.86,9.13,-7.43.1,2.14,-4.08,-9.08,7.82.1,5.05,4.95,-9.17,-8.4,-8.4.1,-8.4.2,...,8.59,3.59,-6.84,-9.03.1,2.82.1,-1.36,-9.08.2,8.3.1,5.68,-4.81,99,99.1,99.2,99.3,99.4,99.5,99.6,-9.42.1,99.7,99.8,99.9,-7.72,99.10,99.11,99.12,99.13,99.14,99.15,99.16,99.17,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
0,"(4.08, 0.8005286678734403)","(-0.29, 0.43322934750185554)","(6.36, 0.39184151058279026)","(4.37, -0.7510255639892701)","(-2.38, 0.4965482674064159)","(-9.66, 1.3432961931388445)","(-0.73, -0.0612070259279587)","(99.0, -0.3237743612334921)","(8.88, -0.19823745516486777)","(9.22, 0.9936480929777722)","(6.75, 1.4566799271177617)","(8.64, 1.3012144112907582)","(4.42, -1.0877251010515785)","(7.43, 1.2514317456915216)","(4.56, -1.0563036110404689)","(-0.97, -1.9899125448279555)","(4.66, -0.6072768678069271)","(99.0, -0.3104917859416468)","(3.3, 0.32686008926629356)","(-1.21, -0.41891097522605275)","(0.87, 1.7311083152933142)","(8.64, 0.7797867615961642)","(8.35, 0.24835997252209588)","(99.0, -0.9514671522874568)","(0.05, 0.5562262170930169)","(7.57, 1.1415338988533767)","(4.71, 2.3987878101441815)","(99.0, 1.2991540525175524)","(99.0, 2.222694023681655)","(6.99, -0.07049299263445825)","(6.5, 1.74992224510506)","(-0.92, 2.3638691079757947)","(7.14, -0.59039691813683)","(9.03, 0.8668717732787246)","(99.0, 2.261913370347303)","(0.73, 2.4141146428309446)","(7.09, -0.7218518781457426)","(3.4, 1.0623731898732283)","(99.0, 1.0168027880060198)","(7.91, 0.9698416940773558)",...,"(-6.7, 1.9543009950003762)","(-3.35, 2.2528617856399573)","(-9.03, 0.4536725326065957)","(4.47, -0.250983641287906)","(4.08, 1.815831286178689)","(-3.83, 1.9626763653541441)","(8.74, -0.3595955837994322)","(1.12, 2.052457739062866)","(0.78, 2.0228751839065793)","(7.52, 0.5691029100669082)","(-5.0, -0.145690299440165)","(2.77, 2.130232611603474)","(8.3, 1.0640032644352242)","(7.77, -0.6853747998429258)","(7.33, 0.148303658853981)","(99.0, 2.0148989153895336)","(7.72, 0.7961996400704514)","(8.98, 1.4679941387914013)","(8.64, 0.4225395299518036)","(8.2, 1.1156224307601355)","(3.93, 1.5649947154776098)","(4.85, 0.8615692554147707)","(4.85, 1.717643868396312)","(6.07, 0.719200827774814)","(8.98, 0.9564441939432216)","(4.51, 0.4188326851103509)","(-0.05, 1.5138991459847972)","(3.69, 1.5590397619836602)","(99.0, 2.618408316367226)","(0.58, 0.7094535996310907)","(2.82, 1.577694199785981)","(-4.95, 1.0835849735092833)","(-0.29, 1.9428106572768482)","(7.86, 1.007045711777407)","(99.0, 0.952979158328613)","(-2.14, 1.2713817197225055)","(3.06, 1.3769586556396487)","(99.0, 0.8783710289854573)","(-4.32, 0.2912066606390206)","(1.07, 1.2328623647330903)"
1,"(99.0, 1.6046591561702788)","(99.0, 0.861554911590811)","(99.0, 0.8685040796848403)","(99.0, -1.6369666010893427)","(9.03, 1.0688476808414245)","(9.27, 2.854268260428134)","(9.03, -0.25302906126795827)","(99.0, -0.7625179381271585)","(99.0, -0.4198541952832693)","(99.0, 2.196511835751656)","(7.33, 3.2271894276680073)","(7.57, 2.842622009339738)","(9.37, -2.4272891977720064)","(6.17, 2.7873032595326905)","(-6.36, -2.4569761175497704)","(-6.89, -4.385010000148334)","(-7.86, -1.294298813479134)","(9.03, -0.6946659449809528)","(9.03, 0.6809492457956798)","(9.03, -0.9498210536799421)","(7.28, 3.8080395952737853)","(99.0, 1.8148787556155335)","(8.25, 0.6334979160352868)","(99.0, -2.1766301915308475)","(99.0, 1.141060677904227)","(7.48, 2.4477655233056326)","(7.28, 5.234618381143485)","(7.28, 2.8551339436901073)","(8.93, 4.906538954114069)","(99.0, -0.09134589424363754)","(99.0, 3.904814472799885)","(7.28, 5.259802919681707)","(99.0, -1.3524229504989025)","(99.0, 1.873846884588342)","(8.98, 5.0819207266416555)","(7.33, 5.302799119640409)","(99.0, -1.713111402391207)","(6.17, 2.2522929588590594)","(9.08, 2.2793092999906825)","(7.33, 2.084385876147447)",...,"(6.46